In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [98]:
df = pd.read_csv(r'../data/fake_job_postings.csv')
df.sample(5)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
172,173,Registrar's in Psychiatry,"AU, QLD, Brisbane",NaN,NaN,Who is Ramberg Associates? Ramberg Associates...,We are seeking Registrar's in Psychiatry for a...,At least 12 months experience at Registrar Lev...,Great Training and Salarys on offer.,0,1,0,Full-time,Associate,NaN,Medical Practice,NaN,0
12266,12267,Marketing Internship,"GB, ,",Appear Here,NaN,Playfair Capital is an early stage technology ...,"We’re on the hunt for talented, energetic indi...",A good eye for detail.Proven management skills...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
17576,17577,Payroll Data Entry Clerk Position - Earn $100...,"US, GA, Abbeville",Clerical,250000-500000,NaN,This job requires you to have the ability to o...,"We are seeking only honest, self-motivated peo...",This position is high paying and you will be p...,0,0,0,Full-time,Not Applicable,High School or equivalent,Staffing and Recruiting,Administrative,1
13176,13177,Cleared Program Manager,"US, DC, Washington",NaN,NaN,Riverside is an internationally recognized eng...,"COMPANY DESCRIPTIONRiverside Technology, inc. ...",Master’s degree required in relevant technical...,Location: Various in Continental U.S.Riverside...,0,1,1,NaN,Director,Master's Degree,Information Technology and Services,Management,0
11165,11166,Turkish Speaking Customer Support Representative,"GR, I, Athens",Reservations,NaN,#URL_c379aa631173ed5b7c345ab3f500a9a053e509138...,As a Turkish Speaking Customer Support Represe...,Required Skills- Native speaker of Langu...,We Offer- Attractive Remuneration Packag...,0,1,1,Full-time,Entry level,NaN,NaN,Customer Service,0


In [99]:
df.shape

(17880, 18)

In [100]:
df.isna().sum()

job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [101]:
df.describe()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent
count,17880.000000,17880.000000,17880.000000,17880.000000,17880.000000
mean,8940.500000,0.042897,0.795302,0.491723,0.048434
std,5161.655742,0.202631,0.403492,0.499945,0.214688
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4470.750000,0.000000,1.000000,0.000000,0.000000
50%,8940.500000,0.000000,1.000000,0.000000,0.000000
75%,13410.250000,0.000000,1.000000,1.000000,0.000000
max,17880.000000,1.000000,1.000000,1.000000,1.000000


In [102]:
df.corr()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent
job_id,1.000000,-0.004559,-0.014539,-0.087025,0.079872
telecommuting,-0.004559,1.000000,-0.019836,0.020345,0.034523
has_company_logo,-0.014539,-0.019836,1.000000,0.233932,-0.261971
has_questions,-0.087025,0.020345,0.233932,1.000000,-0.091627
fraudulent,0.079872,0.034523,-0.261971,-0.091627,1.000000


*Decision*  
  
I have decided to completely drop the column for salary range since there is an overwhelming amount of NaN values and if I were to make a decision on how to fill them then I would be introducing a lot of bias in the data.

In [103]:
df.drop(columns=['salary_range'], inplace=True)

In [104]:
df.isna().sum()

job_id                     0
title                      0
location                 346
department             11547
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

*Decision*  
  
I am going to fill the department NaN values with "Unspecified". Even though there are a lot of values missing I think that it could be potentially less likely for jobs that specify a department to be fraudulent. I could either treat this variable as binary ("has_department", "has_no_department"), or use "Unspecified" and then see if there is maybe another department name that is used for fraudulent jobs.

In [105]:
df['department'] = df['department'].fillna("Unspecified")

In [106]:
df.head()

,job_id,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",Unspecified,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",Unspecified,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [107]:
df['department'].nunique()

1338

Let's see if we can make departments less unique

In [108]:
df['department'] = df['department'].str.lower()

In [109]:
df.head()

,job_id,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",unspecified,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",unspecified,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [110]:
df.loc[df['department'].str.contains('tech'), 'department'] = 'tech'
df.loc[df['department'].str.contains('it'), 'department'] = 'tech'
df.loc[df['department'].str.contains('develop'), 'department'] = 'tech'
df.loc[df['department'].str.contains('data'), 'department'] = 'tech'
df.loc[df['department'].str.contains('cs'), 'department'] = 'tech'
df.loc[df['department'].str.contains('computer'), 'department'] = 'tech'
df.loc[df['department'].str.contains('sales'), 'department'] = 'sales'
df.loc[df['department'].str.contains('eng'), 'department'] = 'engineering'
df.loc[df['department'].str.contains('client'), 'department'] = 'customer'
df.loc[df['department'].str.contains('pr'), 'department'] = 'customer'
df.loc[df['department'].str.contains('relations'), 'department'] = 'customer'
df.loc[df['department'].str.contains('customer'), 'department'] = 'customer'
df.loc[df['department'].str.contains('business'), 'department'] = 'business'
df.loc[df['department'].str.contains('financ'), 'department'] = 'finance'
df.loc[df['department'].str.contains('bank'), 'department'] = 'finance'
df.loc[df['department'].str.contains('account'), 'department'] = 'accounting'
df.loc[df['department'].str.contains('market'), 'department'] = 'marketing'

In [111]:
df['department'].nunique()

790

We have halved the departments, maybe we can work with that

In [112]:
df['title'].nunique()

11231

Maybe apply the same to titles

In [113]:
df['title'] = df['title'].str.lower()

In [114]:
df.loc[df['title'].str.contains('tech'), 'title'] = 'tech'
df.loc[df['title'].str.contains('it'), 'title'] = 'tech'
df.loc[df['title'].str.contains('develop'), 'title'] = 'tech'
df.loc[df['title'].str.contains('data'), 'title'] = 'tech'
df.loc[df['title'].str.contains('cs'), 'title'] = 'tech'
df.loc[df['title'].str.contains('computer'), 'title'] = 'tech'
df.loc[df['title'].str.contains('sales'), 'title'] = 'sales'
df.loc[df['title'].str.contains('eng'), 'title'] = 'engineering'
df.loc[df['title'].str.contains('client'), 'title'] = 'customer'
df.loc[df['title'].str.contains('pr'), 'title'] = 'customer'
df.loc[df['title'].str.contains('relations'), 'title'] = 'customer'
df.loc[df['title'].str.contains('customer'), 'title'] = 'customer'
df.loc[df['title'].str.contains('business'), 'title'] = 'business'
df.loc[df['title'].str.contains('financ'), 'title'] = 'finance'
df.loc[df['title'].str.contains('bank'), 'title'] = 'finance'
df.loc[df['title'].str.contains('account'), 'title'] = 'accounting'
df.loc[df['title'].str.contains('market'), 'title'] = 'marketing'
df.loc[df['title'].str.contains('manager'), 'title'] = 'manager'

In [115]:
df['title'].nunique()

3233

Let's recap our na values

In [116]:
df.isna().sum()

job_id                    0
title                     0
location                346
department                0
company_profile        3308
description               1
requirements           2695
benefits               7210
telecommuting             0
has_company_logo          0
has_questions             0
employment_type        3471
required_experience    7050
required_education     8105
industry               4903
function               6455
fraudulent                0
dtype: int64

*Decision*  
  
I am going to fill the rows that give no location with none.

In [117]:
df['location'] = df['location'].fillna("none,")

In [118]:
df.isna().sum()

job_id                    0
title                     0
location                  0
department                0
company_profile        3308
description               1
requirements           2695
benefits               7210
telecommuting             0
has_company_logo          0
has_questions             0
employment_type        3471
required_experience    7050
required_education     8105
industry               4903
function               6455
fraudulent                0
dtype: int64

Now I am going to process the location. Maybe just use the country of the location, and put in state as a second column, the existence of a state or even the state itself could be an indicator

In [121]:
df['location'] = df['location'].str.lower()
df['country'] = df[["location"]].applymap(lambda str:str.split(",")[0] if "," in str else str)
df['state'] = df[["location"]].applymap(lambda str:str.split(",")[1] if "," in str else 'none')
df.loc[df['state'] == '', 'state'] = 'none'

In [127]:
df.head()

,job_id,title,location,country,state,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,marketing,"us, ny, new york",us,ny,marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,customer,"nz, , auckland",nz,,success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,commissioning machinery assistant (cma),"us, ia, wever",us,ia,unspecified,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,accounting,"us, dc, washington",us,dc,sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,manager,"us, fl, fort worth",us,fl,unspecified,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [128]:
column_to_move = df.pop("country")
df.insert(3, "country", column_to_move)
column_to_move = df.pop("state")
df.insert(4, "state", column_to_move)
df.head()

,job_id,title,location,country,state,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,marketing,"us, ny, new york",us,ny,marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,customer,"nz, , auckland",nz,,success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,commissioning machinery assistant (cma),"us, ia, wever",us,ia,unspecified,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,accounting,"us, dc, washington",us,dc,sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,manager,"us, fl, fort worth",us,fl,unspecified,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [129]:
df.isna().sum()

job_id                    0
title                     0
location                  0
country                   0
state                     0
department                0
company_profile        3308
description               1
requirements           2695
benefits               7210
telecommuting             0
has_company_logo          0
has_questions             0
employment_type        3471
required_experience    7050
required_education     8105
industry               4903
function               6455
fraudulent                0
dtype: int64

In [131]:
df['country'].nunique()

91

In [132]:
df['state'].nunique()

326

I am going to just drop the location column now and use the country and state instead.

In [134]:
df.drop(columns=['location'], inplace=True)

In [166]:
df[df['job_id'] == 11253]['requirements'][11252]

"Bachelor's degree and 10+ years of B2B SaaS sales and sales leadership experienceProven track record building and scaling successful sales organizationsSuperb strategic thinking, strong quantitative analysis, and exceptional communication skillsPassion for mobile, technology, and improving work processes for organizationsStrong understanding of leading markets and trendsAbility to scope and plan projects, as well as manage priorities judiciouslyAbility to think strategically and operate tacticallyAbility to actively communicate, inspire, and motivate all levels of staff"

*Decision*  
  
I will filter the requirements column and see if some keywords appear that I would expect in a real job posting. Words like "experience", "years", "requirements", "qualifications", "must", "degree", "bachelor", "skill". I will then use a binary column to indicate whether one or multiple of these words were present in the description.

In [156]:
df.insert(7, "has_reqs", 0)

In [159]:
df.loc[df['requirements'].str.contains('exp') | df['requirements'].str.contains('year') | df['requirements'].str.contains('qualif') |
       df['requirements'].str.contains('must') | df['requirements'].str.contains('degree') | df['requirements'].str.contains('bachelor') |
       df['requirements'].str.contains('master') | df['requirements'].str.contains('skill'), 'has_reqs'] = 1

In [155]:
df.corr()

,job_id,reqs_present,telecommuting,has_company_logo,has_questions,fraudulent
job_id,1.000000,-0.043293,-0.004559,-0.014539,-0.087025,0.079872
reqs_present,-0.043293,1.000000,-0.016096,0.240892,0.185417,-0.062734
telecommuting,-0.004559,-0.016096,1.000000,-0.019836,0.020345,0.034523
has_company_logo,-0.014539,0.240892,-0.019836,1.000000,0.233932,-0.261971
has_questions,-0.087025,0.185417,0.020345,0.233932,1.000000,-0.091627
fraudulent,0.079872,-0.062734,0.034523,-0.261971,-0.091627,1.000000


*Observation*  
  
It seems like jobs that have no requirements or benefits might mention them in company profile or description, so I will implement the same logic there with the same words for has_reqs.

In [167]:
df.loc[df['company_profile'].str.contains('exp') | df['company_profile'].str.contains('year') | df['company_profile'].str.contains('qualif') |
       df['company_profile'].str.contains('must') | df['company_profile'].str.contains('degree') | df['company_profile'].str.contains('bachelor') |
       df['company_profile'].str.contains('master') | df['company_profile'].str.contains('skill'), 'has_reqs'] = 1

df.loc[df['description'].str.contains('exp') | df['description'].str.contains('year') | df['description'].str.contains('qualif') |
       df['description'].str.contains('must') | df['description'].str.contains('degree') | df['description'].str.contains('bachelor') |
       df['description'].str.contains('master') | df['description'].str.contains('skill'), 'has_reqs'] = 1

In [173]:
df['fraudulent'].sum()